## 1. Load libraries and data

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## 2. Define functions

Funciones

1. **estimate_h_hat**: Permite estimar h_hat dados los valores de theta y x. Ecuación N. 2. 
2. **estimate_p_hat**: Permite estimar p_hat dados los valores de delta y h_hat. Ecuación N. 4.
3. **hh_loss_function**: Permite estimar el costo de la función tomando en cuenta el valor de p, p_hat, h_hat, delta y lambda. Ecuación N. 9. 
4. **gradient_partial**: Permite estimar el gradiente de la función de costo. Ecuación N. 10.
5. **adagrad_update**: Permite actualizar el valor de theta con cada iteración utilizando el algoritmo SGD adagrad. Ecuación N. 11.
6. **half_life_regression**: Modelo HLR que integra todas las funciones anteriores. 

In [66]:
def estimate_h_hat( theta, x ):
    
    '''
    Objetivo:
        - Estimar hat{h} a través de la ecuación N. 2:
          \hat{h} = 2^{Theta \cdot x}
          
    Input:
        - theta : valor de los coeficientes de x
        - x     : variables predictoras
        
    Output:
        - estimated_half_life: hat{h}
    '''
    estimated_h = 2 ** np.dot( theta.T, x )
    
    return estimated_h

In [67]:
def estimate_p_hat( delta, estimated_h ):
    
    '''
    Objetivo:
        - Estimar  hat{p} a través de la ecuación N. 4:
          \hat{p}_{\Theta} = 2^{-\Delta/\hat{h}_{\Theta}}
   
   Input:
       - delta       : tiempo transcurrido desde 
                       la última práctica
       - estimated_h : valor estimado de la capacidad
                       de memoria o half_life ( hat{h} ).
   
   Output:
       - predicted_p : valor estimado de la probabilidad
                       de recordar( hat{p} )
        
    '''
    
    predicted_p = 2 ** ( - delta / estimated_h )
    
    return predicted_p

In [85]:
def hh_loss_function( p, predicted_p, estimated_h, delta, theta, lambda_param = 0.1, alpha_param = 0.01 ):
    
    '''
    Objetivo: 
        - Calcular el valor de pérdida del modelo Half Life Regression
          Ecuación N. 9.
    
    Input:
        - p            : probabilidad de recordar real
        - predicted_p  : probabilidad de recordar predicha
                         mediante la función predict_recall_probability
        - estimated_h  : capacidad de memoria estimada
        - delta        : tiempo transcurrido desde la última práctica
        - theta        : valor de los coeficientes de x
        - lambda_param : parámetro lambda de importancia relativa de la
                         semivida en la función de pérdida
        - alpha_param  : parámetro de regularización L2
        
    Output:
        - Valor de pérdida de la función Half Life Regression
    '''
    
    loss_p              = np.square( p - predicted_p )
    loss_h              = np.square( ( -delta / np.log2( p ) ) - estimated_h )
    regularization_term = lambda_param * np.sum( np.square( theta ) )
    
    loss = loss_p + alpha_param * loss_h + regularization_term  

    return loss

In [86]:
def gradient_partial( p, predicted_p, estimated_h, delta, x, theta, lambda_param = 0.1, alpha_param = 0.01 ):
    
    '''
    Calcula la derivada parcial de la función de pérdida con respecto a cada peso theta_k.

    Input:
    - p            : probabilidad de recordar real
    - predicted_p  : probabilidad de recordar predicha
    - estimated_h  : capacidad de memoria estimada
    - delta        : tiempo transcurrido desde la última práctica
    - x            : vector de características
    - theta        : vector de pesos
    - lambda_param : parámetro lambda de importancia relativa de la semivida en la función de pérdida
    - alpha_param  : parámetro de regularización L2

    Output:
    - gradient : vector de derivadas parciales con respecto a cada theta_k
    '''
    
    term1 = 2 * ( predicted_p - p ) * np.log( 2 ) * predicted_p * ( 2**( -delta / estimated_h ) ) * x
    term2 = 2 * alpha_param * ( estimated_h + delta / np.log2( p ) ) * np.log( 2 ) * estimated_h * x
    term3 = 2 * lambda_param * theta

    gradient = term1 + term2 + term3
    
    return gradient

In [87]:
def adagrad_update( theta, gradient, learning_rate, csg ):
    
    '''
    Actualiza los pesos utilizando el algoritmo AdaGrad.

    Input:
    - theta         : vector de pesos
    - gradient      : vector de derivadas parciales con respecto a cada theta_k
    - learning_rate : tasa de aprendizaje
    - csg           : acumulación de los cuadrados de los gradientes anteriores

    Output:
    - theta_updated : vector de pesos actualizado
    '''
    
    csg_updated = csg + gradient**2
    theta_updated = theta - ( learning_rate / np.sqrt( csg_updated + 1e-8 ) ) * gradient

    return theta_updated, csg_updated

In [88]:
def half_life_regression( X, y, delta, learning_rate=0.01, lambda_param=0.1, alpha_param=0.01, num_iterations = 1000 ):
    '''
    Implementa el modelo de Half-Life Regression.

    Input:
    - X               : Matriz de características (dimensiones: m x n)
    - y               : Vector de etiquetas (dimensiones: m x 1)
    - theta_init      : Vector de pesos iniciales (dimensiones: n x 1)
    - learning_rate   : Tasa de aprendizaje para el algoritmo de optimización (por defecto: 0.01)
    - lambda_param    : Parámetro lambda de importancia relativa de la semivida en la función de pérdida (por defecto: 0.1)
    - alpha_param     : Parámetro de regularización L2 (por defecto: 0.01)
    - num_iterations  : Número de iteraciones para el algoritmo de optimización (por defecto: 1000)

    Output:
    - theta_optimized : Vector de pesos optimizados
    '''

    # Inicialización de variables
    
    m     = X.shape[ 1 ]   # n_rows
    n     = X.shape[ 0 ]   # n_columns
    theta = np.zeros( ( n, 1 ) ) # weights: matriz vacía. Coeficientes. 
    
    # theta = np.random.rand(X.shape[1])
    csg = np.zeros_like(theta)

    cost_list = []
    # Iteraciones de optimización
    for iteration in range(num_iterations):
        # Predicción de la semivida y probabilidad de recordar
        estimated_h = estimate_h_hat(theta, X)       
        predicted_p = estimate_p_hat(delta, estimated_h)

        # Cálculo de la pérdida y el gradiente
        loss     = hh_loss_function(y, predicted_p, estimated_h, delta, theta, lambda_param, alpha_param)
        gradient = gradient_partial(y, predicted_p, estimated_h, delta, X, theta, lambda_param, alpha_param)

        # Actualización de pesos con AdaGrad
        theta, csg = adagrad_update(theta, gradient, learning_rate, csg)

        # Mostrar la pérdida en cada 100 iteraciones
        if iteration % 100 == 0:
            print(f"Iteration {iteration}, Loss: {loss}")
            
        cost_list.append( loss )

    return theta, cost_list

## Primer intento con datos de Duolingo

In [89]:
data      = pd.read_csv( 'subset_1000.csv' )
pred_vars = [ 'right', 'wrong', 'bias', 't' ]

X_train, X_test, Y_train, Y_test = train_test_split( data[ pred_vars ], 
                                                     data[ 'p' ], 
                                                     test_size    = 0.30,
                                                     random_state = 2023 )

t_train = X_train[ 't' ].values
X_train = X_train.values
Y_train = Y_train.values
X_test = X_test.values
Y_test = Y_test.values

X_train = X_train.T
Y_train = Y_train.reshape( 1, X_train.shape[ 1 ] )
t_train = t_train.reshape( 1, X_train.shape[ 1 ] )

X_test = X_test.T
Y_test = Y_test.reshape( 1, X_test.shape[ 1 ] )

In [90]:
theta_optimized, cost_list = half_life_regression(X_train, Y_train, t_train)

Iteration 0, Loss: [[2.69441249e+09 7.37540538e+01 1.70376242e+02 9.46436415e+08
  2.66597506e+01 7.46280790e+01 1.07364127e+10 1.70844490e+07
  2.53931835e+09 6.85275405e+02 4.03974766e+06 5.20247570e+10
  1.48721276e+07 3.60023932e+06 1.69106439e+08 1.43552434e+03
  7.44384112e+06 1.22296864e+03 1.06474829e+08 3.09851513e+05
  4.29282051e+06 1.21306116e+08 2.85250596e+01 4.57751992e+08
  4.38341591e+08 3.67764730e+03 8.13473722e+01 1.89423892e+02
  6.57388621e+07 4.10413328e+01 1.08991475e+02 7.09760674e+06
  2.43009372e+01 1.37983476e+06 3.27345208e+01 1.34438311e+01
  1.09635148e+00 2.03892071e-01 8.29536859e+00 4.00346582e+09
  2.11423184e+03 1.01702478e+06 2.59113784e+02 8.12606442e-01
  2.44621601e+02 6.94611536e+01 3.63105988e+01 3.38637296e+07
  5.82654330e+05 6.50237965e+06 4.93103944e+00 1.45411165e+00
  4.42341364e+06 6.78389531e-01 3.16918504e+07 1.90915268e+07
  5.58505184e-02 8.43019885e+02 2.26710676e+02 6.69471939e+01
  5.40132627e+08 1.20449317e+10 1.56118132e+02 1.94

ValueError: operands could not be broadcast together with shapes (700,700) (4,700) 